In [3]:
import torch

from model import build_transformer
from config import get_config

config = get_config()

# Step 1: Define the model architecture

# device = 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vocab_src_len = 18368
vocab_tgt_len = 6858

model = build_transformer(vocab_src_len, vocab_tgt_len, config["seq_len"], config['seq_len'], d_model=config['d_model']).to(device)


# Step 3: Load the trained weights (state_dict) from the .pt file
model_path = 'cnn_dailymail_weights/tmodel_97.pt'
model.load_state_dict(torch.load(model_path)['model_state_dict'])

<All keys matched successfully>

In [2]:
torch.save(model.state_dict(), 'summerization_model.pth')

In [2]:
dummy_input = torch.randn(1, 400)
onnx_program = torch.onnx.export(model, (dummy_input), 'summerizer_model.onnx')

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



TypeError: Transformer.forward() missing 3 required positional arguments: 'tgt', 'src_mask', and 'tgt_mask'

In [1]:
from summarize import summarize

In [3]:
result = summarize("Fifty five people were killed by a bomb")

Using device: cpu
    SOURCE: Fifty five people were killed by a bomb
 PREDICTED: Gunman shoots kills . Explosion , a   attack on a  included in  among the dead . She . Rep . Police : She . . Latest . Latest attacks . Rep . ET .  

In [4]:
import onnx 
from onnx import helper

onnx_model = onnx.load('./summerizer_model_new.onnx')
print("Model Input: ", [inp.name for inp in onnx_model.graph.input])

Model Input:  ['input_1', 'input_53', 'mask_1', 'mask']


In [3]:
# Define a mapping from old names to new names
name_map = {"input.1": "input_1", "input.53": "input_53", "mask.1": "mask_1"}

# Initialize a list to hold the new inputs
new_inputs = []

# Iterate over the inputs and change their names if needed
for inp in onnx_model.graph.input:
    if inp.name in name_map:
        # Create a new ValueInfoProto with the new name
        new_inp = helper.make_tensor_value_info(name_map[inp.name],
                                                inp.type.tensor_type.elem_type,
                                                [dim.dim_value for dim in inp.type.tensor_type.shape.dim])
        new_inputs.append(new_inp)
    else:
        new_inputs.append(inp)

# Clear the old inputs and add the new ones
onnx_model.graph.ClearField("input")
onnx_model.graph.input.extend(new_inputs)

# Go through all nodes in the model and replace the old input name with the new one
for node in onnx_model.graph.node:
    for i, input_name in enumerate(node.input):
        if input_name in name_map:
            node.input[i] = name_map[input_name]

# Save the renamed ONNX model
onnx.save(onnx_model, 'summerizer_model_new.onnx')

In [5]:
command = "tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default ./tf/saved_model.pb ./tf"
command

'tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default ./tf/saved_model.pb ./tf'

In [7]:
!command

In [1]:
import tensorflowjs as tfjs
import tensorflow

saved_model = 'saved_model.pb'
output_dir = '/'

tfjs.converters.convert_tf_saved_model(saved_model, output_dir)

2024-01-05 17:21:48.544542: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 17:21:48.615075: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 17:21:48.615208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 17:21:48.617226: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 17:21:48.629506: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 17:21:48.630376: I tensorflow/core/platform/cpu_feature_guard.cc:1

OSError: SavedModel file does not exist at: saved_model.pb